# 1. Add a new service endpoint to receive the image and the desired threshold and return  the list of predictions.

The service has been implemented using FastAPI and has been updated in the repository.

# 2. Write a new adapter for ObjectCountRepo to persist data using a relational (MySQL or PostgreSQL) database.

The code in the exsisting repository has been updated to incorporate the use of mySQL in the script.

# 3.Review the rest of the source code and propose some improvements that you would make in the code, the setup instructions, the tests,...

These are my suggestions:

1.The instructions for downloading the model in the README.md file need to be modified. Specifically, "mkdir -p tmp/model/1" should be added before the second step.

2.Adding docstrings and comments within the code can help make it easier to understand.

3.The version "intel/intel-optimized-tensorflow-serving:2.3.0" mentioned in the "Setup and Run Tensorflow Serving" section of the README.md file is not available. Additionally, attempting to pull another version results in an error in prediction.

4.Incorporating a logger into the code can simplify the process of identifying errors.

5.Implementing a try-except block within classes is a good approach.

6. Docker Compose allows for the easy deployment of multi-container Docker applications, simplifying the process of managing and running services.

# 4) Implement at least one of the proposed improvements

In [ ]:
''' Suggestion 1) In README.md file Downloading the model steps to be modified. "mkdir -p tmp/model/1" this should be the second step.'''

wget https://storage.googleapis.com/intel-optimized-tensorflow/models/v1_8/rfcn_resnet101_fp32_coco_pretrained_model.tar.gz
mkdir -p tmp/model/1
tar -xzvf rfcn_resnet101_fp32_coco_pretrained_model.tar.gz -C tmp
rm rfcn_resnet101_fp32_coco_pretrained_model.tar.gz
chmod -R 777 tmp/rfcn_resnet101_coco_2018_01_28
mv tmp/rfcn_resnet101_coco_2018_01_28/saved_model/saved_model.pb tmp/model/1
rm -rf tmp/rfcn_resnet101_coco_2018_01_28



 Added logger and exception handling to the files

5) 	If we want to use multiple models trained internally (not public), what would you change in the setup of the project?

One suggested improvement is to modify the configuration file to enable passing multiple models and then update the object_detector.py file to iterate through the data for each of the different models. Alternatively, we could also utilize the concept of multithreading to reduce the processing time instead of looping through the data.

In [ ]:
'''sample config.py'''


import os

from adapters.count_repo import CountMySQLRepo
from adapters.object_detector import TFSObjectDetector
from domain.actions import CountDetectedObjects

def dev_count_action() -> CountDetectedObjects:
    tfs_host = os.environ.get('TFS_HOST', 'localhost')
    tfs_port = os.environ.get('TFS_PORT', 8501)
    mysql_host = os.environ.get('MYSQL_HOST', 'localhost')
    mysql_port = os.environ.get('MYSQL_PORT', 3306)
    mysql_user = os.environ.get('MYSQL_USER', 'root')
    mysql_password = os.environ.get('MYSQL_PASSWORD', 'password')
    mysql_db = os.environ.get('MYSQL_DB', 'DB')
    models = ['rfcn', 'ssd', 'yolo']
    return CountDetectedObjects(TFSObjectDetector(tfs_host, tfs_port, models),
                                CountMySQLRepo(mysql_host, mysql_port, mysql_user, mysql_password, mysql_db))
    
    
'''sample object_detector.py'''


class TFSObjectDetector(ObjectDetector):
    def __init__(self, host, port, models):
        self.models=models
        self.urls = {model: f"http://{host}:{port}/v1/models/{model}:predict" for model in models}
        self.classes_dicts = {model: self.__build_classes_dict(model) for model in models}

    def predict(self, image: BinaryIO) -> List[Prediction]:
        np_image = self.__to_np_array(image)
        prediction_li=[]
        predict_request = '{"instances" : %s}' % np.expand_dims(np_image, 0).tolist()
        for model in self.models:
            response = requests.post(self.urls[model], data=predict_request)
            predictions = response.json()['predictions'][0]
            pred_value= self.__raw_predictions_to_domain(predictions, self.classes_dicts[model])
            prediction_li.append(pred_value)
        return prediction_li
    @staticmethod
    def __build_classes_dict(model):
         with open('counter/adapters/mscoco_label_map.json') as json_file:
            labels = json.load(json_file)
            return {label['id']: label['display_name'] for label in labels}

    @staticmethod
    def __to_np_array(image: BinaryIO):
        image_ = Image.open(image)
        (im_width, im_height) = image_.size
        return np.array(image_.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

    def __raw_predictions_to_domain(self, raw_predictions: dict, classes_dict: dict) -> List[Prediction]:
        num_detections = int(raw_predictions.get('num_detections'))
        predictions = []
        for i in range(0, num_detections):
            detection_box = raw_predictions['detection_boxes'][i]
            box = Box(xmin=detection_box[1], ymin=detection_box[0], xmax=detection_box[3], ymax=detection_box[2])
            detection_score = raw_predictions['detection_scores'][i]
            detection_class = raw_predictions['detection_classes'][i]
            class_name = classes_dict[detection_class]
            predictions.append(Prediction(class_name=class_name, score=detection_score, box=box))
        return predictions


# 6. Choose one of the following:

  b. Support models for object detection using different deep learning frameworks. If the task seems too big, just lay out the main key points of the proposed solution.



      Let's consider we want to support two different frameworks: TensorFlow and PyTorch.

We could create separate classes for each framework that implement the necessary functionality for object detection. For example, we could create a TensorFlowObjectDetector class and a PyTorchObjectDetector class.

Both classes would have a detect_objects method that takes an image as input and returns a list of detected objects. However, the implementation of this method would be different for each framework.

To allow users to choose which framework to use, we could create a DetectorFactory class that has a create_object_detector method. This method would take a string parameter indicating the desired framework (e.g. "tensorflow" or "pytorch") and return an instance of the appropriate object detector class.


To detect objects using multiple models in parallel, we could use multithreading or multiprocessing. For example, we could create a separate thread or process for each model and feed images to each model in parallel. This would speed up the detection process and allow us to detect objects using multiple models at the same time.

Overall, this approach would allow us to support object detection using multiple deep learning frameworks, and provide users with the flexibility to choose which framework to use.

In [ ]:





'''
The specific implementation of the API can then be created for each framework.
 For example, if we want to support TensorFlow and PyTorch models for object detection, we can create two classes that implement the API for each framework.

Here is an example of how the API could be designed:
'''
import tensorflow as tf
import torch

class ObjectDetector:
    def __init__(self, model):
        self.model = model
        
    def predict(self, image):
        raise NotImplementedError

class TFSObjectDetector(ObjectDetector):
    def __init__(self, model_path):
        model = tf.saved_model.load(model_path)
        super().__init__(model)

    def predict(self, image):
        # Preprocess image and get predictions using TensorFlow model
        processed_image = preprocess_image(image)
        predictions = self.model(processed_image)
        return predictions

class TorchObjectDetector(ObjectDetector):

    def __init__(self, model_path):
        model = torch.load(model_path)
        super().__init__(model)

    def predict(self, image):
        # Preprocess image and get predictions using PyTorch model
        processed_image = preprocess_image(image)
        predictions = self.model(processed_image)
        return predictions

def preprocess_image(image):
    # Preprocess the image according to the input requirements of the model
    # ...
    return "processed_image"

image="abc.jpg"
# Example usage
tf_detector = TFSObjectDetector('models/tf_model')
tf_predictions = tf_detector.predict(image)

torch_detector = TorchObjectDetector('models/torch_model')
torch_predictions = torch_detector.predict(image)

'''
This approach allows us to support multiple deep learning frameworks for object detection without having to modify the rest of the codebase.'''